### Testing Playground for P0 Programs &ndash; Array Literals and Ranges

In [ ]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString
from ST import printSymTab
import time

def runwasm(wasmfile):
    from IPython.core.display import display, Javascript
    display(Javascript("""
    const params = { 
        P0lib: { 
            write: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeln: () => this.append_stream({text: '\\n', name: 'stdout'}),
            read: () => window.prompt()
        }
    }

    fetch('""" + wasmfile + """') // asynchronously fetch file, return Response object
      .then(response => response.arrayBuffer()) // read the response to completion and stores it in an ArrayBuffer
      .then(code => WebAssembly.compile(code)) // compile (sharable) code.wasm
      .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
    // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
    """))

Write, compile and execute P0 programs for testing using the following steps:
1. Define a string constant containing a P0 program to be validated in the cell below.
2. Update the `compileString` parameter to accept the string constant in the cell below.
3. Select `Kernel > Restart & Run All` to view and execute the target WebAssembly program.
   <br>Alternatively, selecting `Cell > Run All` is sufficient to rerun if there are no code changes.

In [ ]:
# Simple array literal assignment
test1 = """
program p
    var a: [1..3] → integer
    a := [3, 7, 11]
    write(a[1])
    write(a[2])
    write(a[3])
"""

# Simple array interval assignment
test2 = """
program p
    var a: [1..3] → integer
    a := [6..8]
    write(a[1])
    write(a[2])
    write(a[3])
"""

# Multiple assignment with array literal at start
test3 = """
program p
    var a: [1..3] → integer
    var b: integer
    var c: boolean
    a, b, c := [3, 7, 11], 42, true
    write(a[1])
    write(a[2])
    write(a[3])
    write(b)
    if c then write(100)
"""

# Multiple assignment with array literal in middle
test4 = """
program p
    var a: [1..3] → integer
    var b: integer
    var c: boolean
    b, a, c := 42, [3, 7, 11], true
    write(a[1])
    write(a[2])
    write(a[3])
    write(b)
    if c then write(100)
"""

# Array literal with mixed types
test5 = """
program p
    var a: [1..3] → integer
    a := [3, true, 11]
"""

# Array literal with wrong type
test6 = """
program p
    var a: [1..3] → integer
    a := [false, true, false]
"""

# Array with non-integer range
test7 = """
program p
    var a: [1..3] → integer
    a := [true..8]
"""

# Array with non-increasing range
test8 = """
program p
    var a: [1..3] → integer
    a := [6..5]
"""

# Array literal of boolean values
test9 = """
program p
    var a: [1..3] → boolean
    a := [false, true, false]
    if a[1] = true then write(100)
    if a[2] = true then write(200)
    if a[3] = true then write(300)
"""

# Single array index assignment (regression test)
test10 = """
program p
    var a: [1..3] → integer
    a[2] := 42
    write(a[2])
"""

# Array direct comparisons
test11 = """
program p
    var a: [1..3] → integer
    var b: [1..3] → integer
    a := [3, 7, 11]
    b := [3, 8, 11]
    if a = [3, 7, 11] then write(100)
    if a = [3, 8, 11] then write(200)
    if [3, 7, 11] = [3, 7, 11] then write(300)
    if [3, 7, 11] = [3, 8, 11] then write(400)
    if a = a then write(500)
    if a = b then write(600)
    if [1, 2, 3] = [1, 2, 3, 4] then write(700)
    if [1, 2, 3, 4] = [1, 2, 3] then write(800)
"""

# Indexing array literals
test12 = """
program p
    write([2,3,4][0])
    write([2,3,4][1])
    write([2,3,4][2])
"""

# Direct assignment of array variables (regression test)
test13 = """
program p
    var a: [1..3] → integer
    var b: [1..3] → integer
    a := [3, 7, 11]
    b := [4, 8, 12]
    a := b
    write(a[1])
    write(a[2])
    write(a[3])
    write(b[1])
    write(b[2])
    write(b[3])
"""

# Update this line to run a specific testcase
compileString(test1, "test.wat")

In [ ]:
!cat -n "test.wat"

In [ ]:
!wat2wasm --enable-bulk-memory test.wat || rm test.wasm

In [ ]:
runwasm("test.wasm")

In [ ]:
time.sleep(1)

The above tests can also be set to run all at once using the following steps:
1. Define testcases using the following format: `(testcase, pass, result)`
   <br>`testcase` is a string such as the above containing a P0 program to be tested.
   <br>`pass` is a Boolean value set to `True` if the program should compile/run without raising an exception.
   <br>`result` is the expected output or exception message when the given program is compiled/run.
2. Set the value of `runall` to `True` below (set to `False` by default to prevent unnecessary runs).
3. Rerun the notebook as above, or run the below cells by selecting `Cell > Run All Below` from the below cell.

Note: the automated test runs should ideally be modified below to capture the output of the `runwasm` function and compare
<br>against `test[2]` to check for the expected result if the program runs successfully, but this has not yet been implemented.

In [ ]:
tests = [
    (test1, True, "3\n7\n11"),
    (test2, True, "6\n7\n8"),
    (test3, True, "3\n7\n11\n42\n100"),
    (test4, True, "3\n7\n11\n42\n100"),
    (test5, False, "array elements must be of the same type"),
    (test6, False, "incompatible assignment"),
    (test7, False, "range start and end must be integers"),
    (test8, False, "range must be increasing"),
    (test9, True, "200"),
    (test10, True, "42"),
    (test11, True, "100\n300\n500"),
    (test12, True, "2\n3\n4"),
    (test13, True, "4\n8\n12\n4\n8\n12")
]

In [ ]:
%%capture errors
runall = False
passing = 0
if runall:
    for index, test in enumerate(tests):
        try:
            compileString(test[0], "test.wat")
            !wat2wasm --enable-bulk-memory test.wat || rm test.wasm
            runwasm("test.wasm")
            # TODO: compare runwasm output to test[2]
            if test[1]: passing += 1
            else: print(f'Test #{index + 1} did not throw an exception')
        except Exception as e:
            if not test[1] and test[2] in str(e): passing += 1
            elif not test[1]: print(f'Test #{index + 1} threw an unexpected exception: {e}')
            else: print(f'Test #{index + 1} threw an unhandled exception: {e}')

In [ ]:
if runall:
    print(f'{passing} out of {len(tests)} tests pass')
    print(errors)
else:
    print("Skipped automated testing")